In [1]:
import json
import requests
import sys
from datetime import datetime, time

sys.path.insert(0, '../..')


from virasana.integracao.due import raspa_due

VIRASANA_URL = "http://localhost/virasana/"
VIRASANA_URL = "http://10.68.64.12/virasana/"

2019-09-10 15:30:47,076 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ../..\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ../..\ajna_commons\flask\access.log


## Realizando consulta no MongoDB Gridfs do AJNA via API virasana

In [2]:
diaapesquisar = datetime.today()
diaapesquisar = datetime(2019, 9, 1)

In [3]:
datainicial = datetime.strftime(datetime.combine(diaapesquisar, time.min), '%Y-%m-%d  %H:%M:%S')
datafinal = datetime.strftime(datetime.combine(diaapesquisar, time.max), '%Y-%m-%d %H:%M:%S')
print(datainicial, datafinal)

2019-09-01  00:00:00 2019-09-01 23:59:59


In [20]:
params = {'query': 
          { 'metadata.dataescaneamento': {'$gte': datainicial, '$lte': datafinal},
            'metadata.contentType': 'image/jpeg',
            'metadata.carga.manifesto.tipomanifesto': 'lce'
          },
          'projection': 
          {'metadata.numeroinformado': 1,
           'metadata.dataescaneamento': 1}
         }

r = requests.post(VIRASANA_URL + "grid_data", json=params)
# print(r.url)

In [21]:
lista = list(r.json())
print(len(lista))
# print(lista)

532


In [22]:
# lista = [['MSCU6656780', '']]
conteineres_ids = {linha['metadata']['numeroinformado']: linha['_id'] for linha in lista}
conteineres = list(conteineres_ids.keys())

# print(conteineres_ids)


## Conectar na API do Pucomex/suiterfb para encontrar e baixar DUE 

In [ ]:
%%time
from selenium import webdriver

GECKO_PATH = 'C:\\Users\\25052288840\\Downloads\\chromedriver.exe'
driver = webdriver.Chrome(GECKO_PATH)
raspa_due.auth_suite_rfb(driver)
try:
    %time conteineres_listadue = raspa_due.get_dues_pos_acd(driver, conteineres)
    # print(conteineres_listadue)
    %time due_detalhe = raspa_due.detalha_dues(driver, conteineres_listadue)
    # print(conteineres_due)
finally:
    driver.close()

    

## Conectar na API do virasana para fazer UPLOAD do arquivo JSON resultante

O arquivo seguirá o formato [{_id: {Documento DUE}}]

In [10]:
print(len(conteineres_listadue))
print(len(due_detalhe))

84
9


In [11]:
for conteiner, dues in list(conteineres_listadue.items())[:40]:
    if dues is not None and len(dues) > 0:
        print(conteiner, dues)
        
for due, conteudo in list(due_detalhe.items())[:10]:
    print(due, json.dumps(conteudo)[:30])

FCIU6444050 ['19BR0011861478']
MSCU3950282 ['19BR0011861478']
MEDU5420837 ['19BR0011861478']
MEDU3614812 ['19BR0011861478']
MEDU5812798 ['19BR0011860170']
MEDU6777412 ['19BR0011860170']
MRKU0811697 ['19BR0012007004']
MSKU6825278 ['19BR0012007004']
19BR0011861478 {"canal": "VERDE", "canalBD": 
19BR0011860170 {"canal": "VERDE", "canalBD": 
19BR0012007004 {"canal": "VERDE", "canalBD": 
19BR0012057222 {"canal": "VERDE", "canalBD": 
19BR0012026483 {"canal": "VERDE", "canalBD": 
19BR0011417721 {"canal": "VERDE", "canalBD": 
19BR0011540540 {"canal": "LARANJA", "canalBD"
19BR0012047839 {"canal": "LARANJA", "canalBD"
19BR0012232121 {"canal": "VERDE", "canalBD": 


In [12]:
with open('dues.txt', 'w') as out:
    json.dump(due_detalhe, out)

In [13]:
with open('dues.txt', 'r') as due_in:
    dues_ = json.load(due_in)

In [14]:
# dues_

In [15]:
def monta_due_ajna(due):
    def get_dados_recinto(recinto_dict):
        result = {}
        result['codigo'] = recinto_dict.get('codigo')
        depositario = recinto_dict.get('depositario')
        if depositario:
            result['depositario'] = depositario.get('depositario')
            result['nome'] = depositario.get('nome')
            result['descricao'] = depositario.get('descricao')
            unidade = depositario.get('unidadeLocalRFB')
            if unidade:
                unidadeLocalRFB = depositario.get('codigo')

        return result 
    
    keys = ['canal', 'chaveAcesso', 'dataProcessamentoTa', 'descricaoTipoItemDue',
            'exportadorEstrangeiro', 'formaExportacao', 'indicadorOEA', 'informacoesComplementares']
    pacote = {}
    for key in keys:
        item = due.get(key)
        if item is not None:
            pacote[key] = item

    declarante = due.get('niDeclarante')
    if declarante:
        pacote['Declarante'] = declarante.get('numero')
        pacote['Nome Declarante'] = declarante.get('nome')

    destino = due.get('paisImportador')
    if destino:
        pacote['PaisImportador'] = destino.get('nome')

    lista_items = due.get('listaInfoItemDue')
    itensDue = []
    for item in lista_items:
        itemDue = {}
        itemDue['descricaoMercadoria'] = item.get('descricaoMercadoria')
        itemDue['exportadorEstrangeiro'] = item.get('exportadorEstrangeiro')
        ncm = item.get('ncm')
        if ncm:
            itemDue['ncm'] = ncm.get('codigo')
        exportador = item.get('niExportador')
        if exportador:
            itemDue['Exportador'] = exportador.get('numero')
            itemDue['NomeExportador'] = exportador.get('nome')
        itensDue.append(itemDue)
    pacote['itens'] = itensDue
    
    for recinto_tipo in ['recintoAduaneiroDespacho',  'recintoAduaneiroEmbarque']:
        pacote[recinto_tipo] = get_dados_recinto(due.get(recinto_tipo))
    ruc = due.get('ruc')
    if ruc:
        pacote['ruc'] = ruc.get('numero')

    return pacote

    


In [16]:
pacote = monta_due_ajna(dues_['19BR0011861478'])
pacote

{'canal': 'VERDE',
 'chaveAcesso': '19TZB022348965',
 'dataProcessamentoTa': '2019-09-02T21:07:18.000+0000',
 'descricaoTipoItemDue': ' Nota Fiscal Eletrônica (NF-e)',
 'exportadorEstrangeiro': False,
 'formaExportacao': 'Por conta própria',
 'indicadorOEA': False,
 'Declarante': '00544628000158',
 'Nome Declarante': 'PRATAPEREIRA COMERCIO IMPORTACAO E EXPORTACAO DE CAFE LTDA',
 'PaisImportador': 'ESTADOS UNIDOS',
 'itens': [{'descricaoMercadoria': 'CAFE CRU, NAO DESCAFEINADO, EM GRAO, ARABICA, COB 6 P/ MELHOR, PENEIRA 16 E ABAIXO, BEBIDA DURA, SAFRA 18/19',
   'exportadorEstrangeiro': False,
   'ncm': '9011110',
   'Exportador': '00544628000158',
   'NomeExportador': 'PRATAPEREIRA COMERCIO IMPORTACAO E EXPORTACAO DE CAFE LTDA'}],
 'recintoAduaneiroDespacho': {'codigo': '8931359',
  'depositario': None,
  'nome': 'BRASIL TERMINAL PORTUARIO S.A.',
  'descricao': None},
 'recintoAduaneiroEmbarque': {'codigo': '8931359',
  'depositario': None,
  'nome': 'BRASIL TERMINAL PORTUARIO S.A.',
 

In [17]:
pacote_carregamento = {}
for conteiner, numeros_dues in conteineres_listadue.items():
    _id = conteineres_ids[conteiner]
    lista_dues = []
    for due in numeros_dues:
        pacote = monta_due_ajna(due_detalhe[due])
        lista_dues.append({'numero': due, **pacote})
    if numeros_dues and len(numeros_dues) > 0:
        pacote_carregamento[_id] = lista_dues
    

In [18]:
pacote_carregamento

{'5d6fc257254139b5c56bf9c4': [{'numero': '19BR0011861478',
   'canal': 'VERDE',
   'chaveAcesso': '19TZB022348965',
   'dataProcessamentoTa': '2019-09-02T21:07:18.000+0000',
   'descricaoTipoItemDue': ' Nota Fiscal Eletrônica (NF-e)',
   'exportadorEstrangeiro': False,
   'formaExportacao': 'Por conta própria',
   'indicadorOEA': False,
   'Declarante': '00544628000158',
   'Nome Declarante': 'PRATAPEREIRA COMERCIO IMPORTACAO E EXPORTACAO DE CAFE LTDA',
   'PaisImportador': 'ESTADOS UNIDOS',
   'itens': [{'descricaoMercadoria': 'CAFE CRU, NAO DESCAFEINADO, EM GRAO, ARABICA, COB 6 P/ MELHOR, PENEIRA 16 E ABAIXO, BEBIDA DURA, SAFRA 18/19',
     'exportadorEstrangeiro': False,
     'ncm': '9011110',
     'Exportador': '00544628000158',
     'NomeExportador': 'PRATAPEREIRA COMERCIO IMPORTACAO E EXPORTACAO DE CAFE LTDA'}],
   'recintoAduaneiroDespacho': {'codigo': '8931359',
    'depositario': None,
    'nome': 'BRASIL TERMINAL PORTUARIO S.A.',
    'descricao': None},
   'recintoAduaneiroEm

In [19]:
r = requests.post(VIRASANA_URL + "dues/update", json=pacote_carregamento)
print(r.status_code)
print(r.text)


201
{
  "status": "DUEs inseridas/atualizadas"
}

